In [1]:
# for  RFM analysis & segmentation
import pandas as pd
import numpy as np
import datetime as dt


# --- STEP 1: LOAD DATA ---
# Replace this with your actual SQL connection logic
# df = pd.read_sql("SELECT * FROM cleaned_transactions", connection)

# Dummy Data for demonstration
data = {
    'CustomerID': [1, 1, 2, 2, 3, 4, 5],
    'TransactionDate': pd.to_datetime(['2023-10-01', '2023-12-01', '2023-11-15', '2023-12-20', '2023-01-10', '2023-12-22', '2023-05-05']),
    'Amount': [100, 150, 50, 300, 20, 500, 80]
}
df = pd.DataFrame(data)

# --- STEP 2: CALCULATE RFM ---
NOW = dt.datetime(2023, 12, 25) # Reference date

rfm = df.groupby('CustomerID').agg({
    'TransactionDate': lambda x: (NOW - x.max()).days, # Recency
    'CustomerID': 'count',                             # Frequency
    'Amount': 'sum'                                    # Monetary
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']

# --- STEP 3: ASSIGN SCORES (1-5) ---
# Lower recency is better (higher score)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# --- STEP 4: SEGMENTATION LOGIC ---
def segment_customer(df):
    if df['R_Score'] >= 4 and df['M_Score'] >= 4:
        return 'Champions'
    elif df['R_Score'] <= 2:
        return 'Hibernating'
    else:
        return 'Regulars'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)
print("--- RFM Segmentation Results ---")
print(rfm.head())





--- RFM Segmentation Results ---
            Recency  Frequency  Monetary R_Score F_Score M_Score      Segment
CustomerID                                                                   
1                24          2       250       3       4       3     Regulars
2                 5          2       350       4       5       4    Champions
3               349          1        20       1       1       1  Hibernating
4                 3          1       500       5       2       5    Champions
5               234          1        80       2       3       2  Hibernating


In [2]:
# Validation: Compare average Monetary value per segment
validation = rfm.groupby('Segment')['Monetary'].mean().sort_values(ascending=False)
print("\nValidation - Average Spend by Segment:")
print(validation) 


Validation - Average Spend by Segment:
Segment
Champions      425.0
Regulars       250.0
Hibernating     50.0
Name: Monetary, dtype: float64


In [3]:
!pip install mlxtend

  Using cached mlxtend-0.24.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached scikit_learn-1.8.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached matplotlib-3.10.8-cp312-cp312-win_amd64.whl.metadata (52 kB)
Using cached mlxtend-0.24.0-py3-none-any.whl (1.4 MB)
Using cached matplotlib-3.10.8-cp312-cp312-win_amd64.whl (8.1 MB)
Using cached scikit_learn-1.8.0-cp312-cp312-win_amd64.whl (8.0 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.7
    Uninstalling matplotlib-3.10.7:
      Successfully uninstalled matplotlib-3.10.7



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from mlxtend.frequent_patterns import apriori, association_rules

# Sample Transaction Data
basket_data = {
    'TransactionID': [1, 1, 2, 2, 3, 4, 4, 4],
    'Item': ['Milk', 'Bread', 'Milk', 'Diapers', 'Milk', 'Bread', 'Milk', 'Diapers']
}
df_basket = pd.DataFrame(basket_data)

# --- STEP 1: ONE-HOT ENCODING ---
basket = (df_basket.groupby(['TransactionID', 'Item'])['Item']
          .count().unstack().reset_index().fillna(0)
          .set_index('TransactionID'))

def encode_units(x):
    return 1 if x >= 1 else 0

basket_sets = basket.applymap(encode_units)

# --- STEP 2: APPLY APRIORI ---
frequent_itemsets = apriori(basket_sets, min_support=0.5, use_colnames=True)

# --- STEP 3: GENERATE RULES ---
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print("\n--- Market Basket Association Rules ---")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


--- Market Basket Association Rules ---
  antecedents consequents  support  confidence  lift
0      (Milk)     (Bread)      0.5         0.5   1.0
1     (Bread)      (Milk)      0.5         1.0   1.0
2      (Milk)   (Diapers)      0.5         0.5   1.0
3   (Diapers)      (Milk)      0.5         1.0   1.0


C:\Users\lenovo\AppData\Local\Temp\ipykernel_2452\3009325524.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.applymap(encode_units)
C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:175: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
C:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\association_rules.py:186: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)
